In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES']='7'

import keras 
from keras.layers.merge import Grad 
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger


Using TensorFlow backend.


In [2]:
#load the data 
import h5py 
import numpy as np 
data_prefix="/srv/scratch/annashch/deeplearning/gradient_exp/"
train_hdf5=h5py.File(data_prefix+"train_data.hdf5",'r')
train_X=train_hdf5['X']['sequence']
train_Y=train_hdf5['Y']['output']

valid_hdf5=h5py.File(data_prefix+'valid_data.hdf5','r')
valid_X=valid_hdf5['X']['sequence']
valid_Y=valid_hdf5['Y']['output']

test_hdf5=h5py.File(data_prefix+"test_data.hdf5",'r')
test_X=test_hdf5['X']['sequence']
test_Y=test_hdf5['Y']['output']

weights_gata=np.load('weights_gata_train.npy')
Y_grads_gata=np.load('gradients_gata_train.npy')
weights_tal=np.load('weights_tal_train.npy')
Y_grads_tal=np.load('gradients_tal_train.npy')
weights_gata_valid=np.load('weights_gata_valid.npy')
Y_grads_gata_valid=np.load('gradients_gata_valid.npy')
weights_tal_valid=np.load('weights_tal_valid.npy')
Y_grads_tal_valid=np.load('gradients_tal_valid.npy')
weights_gata_test=np.load('weights_gata_test.npy')
Y_grads_gata_test=np.load('gradients_gata_test.npy')
weights_tal_test=np.load('weights_tal_test.npy')
Y_grads_tal_test=np.load('gradients_tal_test.npy')

In [3]:
#build the model -- without gradient reward (this is our baseline)
input_shape=(200,4)
pool_size=10 
np.random.seed(1234)
inp = keras.layers.Input(shape=input_shape)
conv1 = keras.layers.Convolution1D(nb_filter=10, filter_length=10, subsample_length=1)(inp)
relu1 = keras.layers.Activation("relu")(conv1)
#pool1=keras.layers.pooling.AveragePooling1D(pool_size=4)(relu1)
conv2 = keras.layers.Convolution1D(nb_filter=10, filter_length=30, subsample_length=1)(relu1)
relu2 = keras.layers.Activation("relu")(conv2)
gap = keras.layers.pooling.GlobalAveragePooling1D()(relu2)
dense = keras.layers.Dense(2)(gap)
sigmoid_out = keras.layers.Activation("sigmoid")(dense)
model = keras.models.Model(input=inp, output=sigmoid_out)
model.compile(optimizer="adam", loss="binary_crossentropy",metrics=["binary_accuracy"])

/users/annashch/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(strides=1, filters=10, kernel_size=10)`
/users/annashch/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(strides=1, filters=10, kernel_size=30)`
/users/annashch/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("in...)`


In [4]:
#train the model 
model_output_path="benchmarkModel"
#checkpointer = ModelCheckpoint(filepath=model_output_path)
earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
tensorboard = TensorBoard( write_images=True)
csv_logger = CSVLogger('training.benchmark.log')

model.fit(x=np.asarray(train_X), 
          y=np.asarray(train_Y),
          callbacks=[earlystopper,csv_logger,tensorboard],
          batch_size=250,
          epochs=50,
          validation_data=tuple([valid_X,valid_Y]),
          verbose=1)

Train on 2400 samples, validate on 300 samples
Epoch 1/50
2400/2400 [==============================] - 1s - loss: 0.6764 - binary_accuracy: 0.6110 - val_loss: 0.6559 - val_binary_accuracy: 0.6500
Epoch 2/50
2400/2400 [==============================] - 0s - loss: 0.6397 - binary_accuracy: 0.6679 - val_loss: 0.6584 - val_binary_accuracy: 0.6500
Epoch 3/50
2400/2400 [==============================] - 0s - loss: 0.6389 - binary_accuracy: 0.6679 - val_loss: 0.6480 - val_binary_accuracy: 0.6500
Epoch 4/50
2400/2400 [==============================] - 0s - loss: 0.6368 - binary_accuracy: 0.6679 - val_loss: 0.6479 - val_binary_accuracy: 0.6500
Epoch 5/50
2400/2400 [==============================] - 0s - loss: 0.6357 - binary_accuracy: 0.6679 - val_loss: 0.6485 - val_binary_accuracy: 0.6500
Epoch 6/50
2400/2400 [==============================] - 0s - loss: 0.6354 - binary_accuracy: 0.6679 - val_loss: 0.6483 - val_binary_accuracy: 0.6500
Epoch 7/50
2400/2400 [==============================] - 0s 

In [29]:
#build the model with gradient reward 
reload(keras.losses)
from keras.layers.merge import Grad
input_shape=(200,4)
pool_size=20 
np.random.seed(1234)
inp = keras.layers.Input(shape=input_shape)
conv1 = keras.layers.Convolution1D(nb_filter=10, filter_length=10, subsample_length=1)(inp)
relu1 = keras.layers.Activation("relu")(conv1)
#pool1=keras.layers.pooling.MaxPooling1D(pool_size=4)(relu1)
conv2 = keras.layers.Convolution1D(nb_filter=10, filter_length=30, subsample_length=1)(relu1)
relu2 = keras.layers.Activation("relu")(conv2)
gap = keras.layers.pooling.GlobalAveragePooling1D()(relu2)
dense = keras.layers.Dense(2)(gap)
sigmoid_out = keras.layers.Activation("sigmoid")(dense)
grad_layer1 = Grad(task_index=0)([inp, dense])
grad_layer2 = Grad(task_index=1)([inp, dense])
model_grad = keras.models.Model(input=inp, output=[sigmoid_out, grad_layer1,grad_layer2])
model_grad.compile(sample_weight_mode=[None,"temporal","temporal"],
                   optimizer="adam", 
                   loss=["binary_crossentropy",
                         keras.losses.get_positionwise_cosine_1d(pool_size=pool_size),
                         keras.losses.get_positionwise_cosine_1d(pool_size=pool_size)],
                   metrics=["binary_accuracy"])

/users/annashch/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(strides=1, filters=10, kernel_size=10)`
/users/annashch/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(strides=1, filters=10, kernel_size=30)`
/users/annashch/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


In [30]:
#train the model 
model_output_path="gradModel"
#checkpointer = ModelCheckpoint(filepath=model_output_path)
earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
tensorboard = TensorBoard( write_images=True)
csv_logger = CSVLogger('training.grad.log')
model_grad.fit(x=np.asarray(train_X), 
          y=[np.asarray(train_Y),Y_grads_gata,Y_grads_tal],
               sample_weight=[None,weights_gata*100,weights_tal*100],
          callbacks=[csv_logger],
               validation_data=tuple([valid_X,
                                      [np.asarray(valid_Y),Y_grads_gata_valid,Y_grads_tal_valid],
                                      [None,weights_gata_valid,weights_tal_valid]
                                     ]),
          batch_size=250,
          epochs=50,
          verbose=1)

Train on 2400 samples, validate on 300 samples
Epoch 1/50
2400/2400 [==============================] - 0s - loss: -46.3965 - activation_42_loss: 0.6764 - grad_25_loss: -19.8506 - grad_26_loss: -27.2223 - activation_42_binary_accuracy: 0.6110 - grad_25_binary_accuracy: 0.9376 - grad_26_binary_accuracy: 0.8996 - val_loss: -0.1499 - val_activation_42_loss: 0.6559 - val_grad_25_loss: -0.3271 - val_grad_26_loss: -0.4787 - val_activation_42_binary_accuracy: 0.6500 - val_grad_25_binary_accuracy: 0.9378 - val_grad_26_binary_accuracy: 0.9035
Epoch 2/50
2400/2400 [==============================] - 0s - loss: -74.5745 - activation_42_loss: 0.6397 - grad_25_loss: -30.9093 - grad_26_loss: -44.3049 - activation_42_binary_accuracy: 0.6679 - grad_25_binary_accuracy: 0.9376 - grad_26_binary_accuracy: 0.8996 - val_loss: -0.0708 - val_activation_42_loss: 0.6584 - val_grad_25_loss: -0.3117 - val_grad_26_loss: -0.4175 - val_activation_42_binary_accuracy: 0.6500 - val_grad_25_binary_accuracy: 0.9378 - val_g

In [33]:
model_grad.total_loss


<tf.Tensor 'add_233:0' shape=() dtype=float32>